# word2vec 개선 (1)

**3장의 simple word2vec에 두 가지 개선 방안 추가**

1. Embedding이라는 새로운 계층 도입

2. 네거티브 샘플링이라는 새로운 손실 함수 도입

3장에서의 CBOW 모델은 다루는 어휘의 수가 총 7개이기에 문제없이 작동한다. 하지만 거대한 말뭉치를 다루게 되면 문제가 발생한다.  

어휘가 100만 개, 은닉층의 뉴런이 100개인 CBOW를 생각해보자. 입력층과 출력층에는 각 100만 개의 뉴런이 존재한다.

이 때 다음의 두 계산이 병목된다. 

- **입력층의 원핫 표현과 가중치 행렬 $W_{in}$의 곱 계산**
  - 원핫 표현과 관련된 문제
  - `Embedding`이라는 새로운 계층 도입
  
        
- **은닉층과 가중치 행렬 $W_{out}$의 곱 및 Softmax 계층의 계산**
  - 은닉층 이후의 계산 문제
  - 은닉층과 가중치 행렬 $W_{out}$의 곱의 계산량이 상당함
  - `negative sampling` 이라는 새로운 손실 함수 도입

## Embedding 계층

어휘의 수가 100만 개, 은닉층의 뉴런이 100개인 경우를 가정해보자.

$c * W_{in} = h$ 일 때, 형상은 다음과 같다.

- c.shape = (1, 1000000)
- $W_{in}$.shape = (1000000, 100)
- h.shape = (1,100)

**> 이는 결과적으로 $W_{in}$ 행렬에서 특정 행을 추출하는 것 뿐이다.**   
여기서 말하는 "특정 행"이라 함은 $W_{in}$ 행렬에서 input c 단어의 id에 해당하는 인덱스 행을 말함

#### 즉, 원핫 표현과 MatMul 계층의 행렬곱 연산은 사실 상 불필요한 작업임!
---

그러면 가중치 매개변수로부터 `단어 ID에 해당하는 행(벡터)`를 추출하는 계층을 만들어보자!<br>
<u>여기서의 계층이 바로 Embedding 계층!</u>

**참고) Embedding은 단어 임베딩이라는 용어에서 유래했다. 즉, embedding 계층에 단어 임베딩 (분산 표현)을 저장하는 것임.

## Embedding 계층 구현

In [2]:
import numpy as np

W = np.arange(21).reshape(7,3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [6]:
# 단일 행 뽑기
# idx = 2
display(W[2])
# idx = 5
display(W[5])

array([6, 7, 8])

array([15, 16, 17])

In [7]:
# idx 여러개 한번에 추출
idx = np.array([1,0,3,0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [9]:
class Embedding: 
    def __init__(self,W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None # 단어 ID 인덱스를 배열로 저장, 미니배치를 고려할 때 여러개도 추출될 수 있도록 구현
        
    # 가중치 W의 특정 행을 추출
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx] 
        return out
    
    # 반대로 역전파에서는 반대로 출력층에서의 기울기를 W의 idx 행에 할당
    def backward(self, dout):
        dW, = self.grads # 가중치 기울기 dW를 꺼낸다
        dW[...] = 0 # dW의 형상을 유지한 채 원소만 0으로 덮어쓴다
        
        # idx 중복을 고려하지 않은 방법
        # dW[self.idx] = dout # 앞 층에서 전해진 기울기 dout을 idx 번째 행에 할당한다. 
        
        # idx 중복 고려 : 할당 x 더하기 !
        # Method #1 느리당
        #for i, word_id in enumerate(self.idx):
        #    dW[word_id] += dout[i]
        
        # Method #2 효율적. np.add.at(A,idx,B) = B를 A의 idx번째 행에 더해준다.
        np.add.at(dW, self.idx, dout)
        
        return None

(주의) 역전파 과정에서 idx의 중복을 고려하여 값을 '할당'하는 것이 아니라 값을 **더하기** 한다.